In [ ]:
#
# @ 2021. TU Dortmund University,
# Institute of Energy Systems, Energy Efficiency and Energy Economics,
# Research group Distribution grid planning and operation
#

In [24]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [25]:
cols = ["HP_ID", "W_ID", "ST_WOTAG", "W_SZ", "W_AZ", "wegmin_imp1", "zweck", "zweck_mop", "wegkm", "km_routing", "tempo", "feiertag", "hvm_imp", "min_altern_auto", "HP_BKAT", "W_ZWDE", "W_ZWDP", "W_ZWDF","W_FOLGETAG", "W_SO1", "MODE", "RegioStaR7", "RegioStaR4", "RegioStaRGem7", "RegioStaRGem5"] 

In [26]:
data = pd.read_csv(r"C:\Users\User\Desktop\MiD\CSV\MiD2017_Wege.csv", usecols=cols, sep=";")

In [27]:
# Spalten in gewünschter Ordnung
data = data[cols]

In [28]:
data.head()

,HP_ID,W_ID,ST_WOTAG,W_SZ,W_AZ,wegmin_imp1,zweck,zweck_mop,wegkm,km_routing,tempo,feiertag,hvm_imp,min_altern_auto,HP_BKAT,W_ZWDE,W_ZWDP,W_ZWDF,W_FOLGETAG,W_SO1,MODE,RegioStaR7,RegioStaR4,RegioStaRGem7,RegioStaRGem5
0,1001431,1,1,6:30:00,7:00:00,30,1,1,"21,85","20,946","43,7",0,4,70706,1,7704,7704,7704,0,1,2,77,22,77,55
1,1001431,2,1,12:10:00,12:30:00,20,5,3,",39",70706,"1,17",0,1,70706,1,7704,706,7704,0,809,2,77,22,77,55
2,1001431,3,1,16:15:00,16:45:00,30,8,5,"21,85","22,653","43,7",0,4,70706,1,7704,7704,7704,0,809,2,77,22,77,55
3,1001432,1,1,8:00:00,8:25:00,25,7,4,"2,45",70706,"5,88",0,1,70706,3,7704,7704,704,0,1,2,77,22,77,55
4,1001432,2,1,8:25:00,8:45:00,20,8,5,"2,45",70706,"7,35",0,1,70706,3,7704,7704,7704,0,809,2,77,22,77,55


In [29]:
# Umwandeln der String Repräsentation von Zahlen zu floats

data["wegkm"] = data["wegkm"].apply(lambda x: float(x.replace(",", ".")))
data["km_routing"] = data["km_routing"].apply(lambda x: float(x.replace(",", ".")))
data["tempo"] = data["tempo"].apply(lambda x: float(x.replace(",", ".")))

In [30]:
trips_not_from_home = len(data[data["W_SO1"] == 2])
number_first_trips = len(data[data["W_ID"] == 1])

trips_not_from_home / number_first_trips

0.05337916215774354

In [32]:
# Triptagebücher aussortieren dessen erste Wege nicht Zuhause starten

# Trips bei denen Abfahrtszeit + Wegdauer nicht Ankunftszeit ergibt und der Trip nicht in den Folgetag hineinreicht
filt = data["W_SO1"] == 2
ids_to_drop = data[filt]["HP_ID"]
filt = data["HP_ID"].isin(ids_to_drop)
rows_to_drop = data[filt].index
data.drop(index=rows_to_drop, inplace=True)
data.reset_index(drop=True, inplace=True)

In [33]:
# Triptagebücher aussortieren, bei welchen entweder Start oder Ankunftszeit fehlen

filt = (data["W_SZ"] == ' ') | (data["W_AZ"] == ' ')
ids_to_drop = data[filt]["HP_ID"]
filt = data["HP_ID"].isin(ids_to_drop)
rows_to_drop = data[filt].index
data.drop(index=rows_to_drop, inplace=True)
data.reset_index(drop=True, inplace=True)

In [34]:
def time_conversion(t):
    h, m, s = str(t).split(":")
    return int(int(h) * 60 + int(m) + int(s) / 60)

In [35]:
data["Departure"] = data["W_SZ"].apply(lambda x: time_conversion(x))
data["Arrival"] = data["W_AZ"].apply(lambda x: time_conversion(x))

In [36]:
# Triptagebücher aussortieren, bei welchen Start / Ankunftszeiten und angegebene Wegdauern nicht übereinstimmen

# Trips bei denen Abfahrtszeit + Wegdauer nicht Ankunftszeit ergibt und der Trip nicht in den Folgetag hineinreicht
filt = (data["Arrival"] != data["Departure"] + data["wegmin_imp1"]) & (data["W_FOLGETAG"] == 0)
ids_to_drop = data[filt]["HP_ID"]
filt = data["HP_ID"].isin(ids_to_drop)
rows_to_drop = data[filt].index
data.drop(index=rows_to_drop, inplace=True)
data.reset_index(drop=True, inplace=True)

In [37]:
# Umgang mit Wegen welche nicht mit PKW / durch Ev substituierbaren Fahrzeugen zurückgelegt wurden

#Lösche alle Wegetagebücher in welchen Fahrten vorkommen die nicht mit entsprechendem Fahrzeug durchgeführt wurden

filt = (data["hvm_imp"] < 3) | (data["hvm_imp"] > 4)
ids_to_drop = data[filt]["HP_ID"]
filt = data["HP_ID"].isin(ids_to_drop)
rows_to_drop = data[filt].index
data.drop(index=rows_to_drop, inplace=True)
data.reset_index(drop=True, inplace=True) 

# mögliche Alternative 2: ÖPV Fahrten ersetzen -> Distanz = km_routing , Wegmin = min_altern_auto
    
data

,HP_ID,W_ID,ST_WOTAG,W_SZ,W_AZ,wegmin_imp1,zweck,zweck_mop,wegkm,km_routing,tempo,feiertag,hvm_imp,min_altern_auto,HP_BKAT,W_ZWDE,W_ZWDP,W_ZWDF,W_FOLGETAG,W_SO1,MODE,RegioStaR7,RegioStaR4,RegioStaRGem7,RegioStaRGem5,Departure,Arrival
0,1002822,1,1,5:50:00,5:55:00,5,1,1,0.19,0.649,2.28,0,4,70706,1,7704,7704,7704,0,1,4,77,22,77,55,350,355
1,1002822,2,1,14:30:00,14:55:00,25,5,3,23.28,23.836,55.87,0,4,70706,1,7704,601,7704,0,809,4,77,22,77,55,870,895
2,1002822,3,1,15:30:00,15:55:00,25,8,5,23.28,24.417,55.87,0,4,70706,1,7704,7704,7704,0,809,4,77,22,77,55,930,955
3,1002911,1,1,8:50:00,9:00:00,10,6,3,4.09,16.394,24.54,0,4,70706,7,7704,7704,7704,0,1,2,77,22,77,55,530,540
4,1002911,2,1,9:15:00,9:50:00,35,6,3,26.60,25.893,45.60,0,4,70706,7,7704,7704,7704,0,809,2,77,22,77,55,555,590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290003,99991831,4,7,17:00:00,17:15:00,15,8,5,5.70,1.378,22.80,0,4,70706,7,7704,7704,7704,0,809,2,75,22,74,53,1020,1035
290004,99994852,1,7,17:30:00,17:45:00,15,7,4,2.85,4.149,11.40,0,4,70706,7,7704,7704,713,0,1,2,73,11,75,53,1050,1065
290005,99994852,2,7,19:00:00,19:15:00,15,8,5,2.85,4.149,11.40,0,4,70706,7,7704,7704,7704,0,809,2,73,11,75,53,1140,1155
290006,99998511,1,1,7:30:00,7:45:00,15,1,1,3.99,9995.000,15.96,0,4,70706,1,7704,7704,7704,0,1,2,76,21,75,53,450,465


In [38]:
data.drop(columns=["km_routing", "hvm_imp", "min_altern_auto", "W_SO1"], inplace=True)

In [39]:
data.head()

,HP_ID,W_ID,ST_WOTAG,W_SZ,W_AZ,wegmin_imp1,zweck,zweck_mop,wegkm,tempo,feiertag,HP_BKAT,W_ZWDE,W_ZWDP,W_ZWDF,W_FOLGETAG,MODE,RegioStaR7,RegioStaR4,RegioStaRGem7,RegioStaRGem5,Departure,Arrival
0,1002822,1,1,5:50:00,5:55:00,5,1,1,0.19,2.28,0,1,7704,7704,7704,0,4,77,22,77,55,350,355
1,1002822,2,1,14:30:00,14:55:00,25,5,3,23.28,55.87,0,1,7704,601,7704,0,4,77,22,77,55,870,895
2,1002822,3,1,15:30:00,15:55:00,25,8,5,23.28,55.87,0,1,7704,7704,7704,0,4,77,22,77,55,930,955
3,1002911,1,1,8:50:00,9:00:00,10,6,3,4.09,24.54,0,7,7704,7704,7704,0,2,77,22,77,55,530,540
4,1002911,2,1,9:15:00,9:50:00,35,6,3,26.60,45.60,0,7,7704,7704,7704,0,2,77,22,77,55,555,590


In [40]:
# Umbenennen der Spaltennamen
data.columns=["ID", "Trip_no", "Type_day", "Departure(hh:mm)", "Arrival(hh:mm)", "Trip_duration", "Whyto", "Whyto_mop", "Distance", "Av_speed", "Holiday", "Work_status", "W_ZWDE", "W_ZWDP", "W_ZWDF", "Overnight", "Mode", "RegioStaR7", "RegioStaR4", "RegioStaRGem7", "RegioStaRGem5", "Departure", "Arrival"]

In [41]:
# Neuanordnung der Spalten
data = data[["ID", "Trip_no", "Type_day", "Departure(hh:mm)", "Arrival(hh:mm)", "Departure", "Arrival", "Trip_duration", "Whyto", "Whyto_mop", "Distance", "Av_speed", "Holiday", "Work_status", "W_ZWDE", "W_ZWDP", "W_ZWDF", "Overnight", "RegioStaR7", "RegioStaR4", "RegioStaRGem7", "RegioStaRGem5", "Mode"]]

In [42]:
data.head()

,ID,Trip_no,Type_day,Departure(hh:mm),Arrival(hh:mm),Departure,Arrival,Trip_duration,Whyto,Whyto_mop,Distance,Av_speed,Holiday,Work_status,W_ZWDE,W_ZWDP,W_ZWDF,Overnight,RegioStaR7,RegioStaR4,RegioStaRGem7,RegioStaRGem5,Mode
0,1002822,1,1,5:50:00,5:55:00,350,355,5,1,1,0.19,2.28,0,1,7704,7704,7704,0,77,22,77,55,4
1,1002822,2,1,14:30:00,14:55:00,870,895,25,5,3,23.28,55.87,0,1,7704,601,7704,0,77,22,77,55,4
2,1002822,3,1,15:30:00,15:55:00,930,955,25,8,5,23.28,55.87,0,1,7704,7704,7704,0,77,22,77,55,4
3,1002911,1,1,8:50:00,9:00:00,530,540,10,6,3,4.09,24.54,0,7,7704,7704,7704,0,77,22,77,55,2
4,1002911,2,1,9:15:00,9:50:00,555,590,35,6,3,26.60,45.60,0,7,7704,7704,7704,0,77,22,77,55,2


In [43]:
def drop_by_filter(data, filt):
    ids_to_drop = data[filt]["ID"]
    filt = data["ID"].isin(ids_to_drop)
    rows_to_drop = data[filt].index
    data.drop(index=rows_to_drop, inplace=True)
    data.reset_index(drop=True, inplace=True)  
    return data

In [44]:
# Distanz 
    # 9994 -> unplausibler Wert
    # 9999 -> keine Angabe
    # 70703 -> Weg ohne Detailerfassung

# Bereinigung um Triptagebücher mit Trips mit unplausiblen Werten, fehlenden Angaben oder fehlender Detailerfassung
filt = (data["Distance"] == 9994) | (data["Distance"] == 9999) | (data["Distance"] == 70703)
data = drop_by_filter(data, filt)

# Av_speed:
    # 9994 -> unplausibel
    # 9999 -> Wert nicht berechenbar
    # 70701 -> bei rbW nicht bestimmbar
    # 70703 -> Weg ohne Detailerfassung

# Bereinigung um Triptagebücher mit Trips mit unplausiblen Werten, fehlenden Angaben oder fehlender Detailerfassung
filt = (data["Av_speed"] == 9994) | (data["Av_speed"] == 9999) | (data["Av_speed"] == 70703)
data = drop_by_filter(data, filt)


C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [45]:
data.head()

,ID,Trip_no,Type_day,Departure(hh:mm),Arrival(hh:mm),Departure,Arrival,Trip_duration,Whyto,Whyto_mop,Distance,Av_speed,Holiday,Work_status,W_ZWDE,W_ZWDP,W_ZWDF,Overnight,RegioStaR7,RegioStaR4,RegioStaRGem7,RegioStaRGem5,Mode
0,1002822,1,1,5:50:00,5:55:00,350,355,5,1,1,0.19,2.28,0,1,7704,7704,7704,0,77,22,77,55,4
1,1002822,2,1,14:30:00,14:55:00,870,895,25,5,3,23.28,55.87,0,1,7704,601,7704,0,77,22,77,55,4
2,1002822,3,1,15:30:00,15:55:00,930,955,25,8,5,23.28,55.87,0,1,7704,7704,7704,0,77,22,77,55,4
3,1007211,1,1,10:00:00,10:19:00,600,619,19,5,3,3.80,12.00,0,7,7704,602,7704,0,77,22,77,55,2
4,1007211,2,1,10:29:00,10:35:00,629,635,6,4,3,1.90,19.00,0,7,501,7704,7704,0,77,22,77,55,2


In [46]:
# ersetzen Rückweg mit tatsächlichem Zielzustand
for i in range(len(data)):
    if data.at[i, "Whyto"] == 9:
        data.at[i, "Whyto"] = data.at[i-2, "Whyto"]

In [47]:
# Zustände neuzuordnen

# Erreichen des Arbeitsplatzes → Arbeit                    1 : 1  
# dienstlich/geschäftlich → Arbeit                         2 : 1
# Erreichen der Ausbildungsstätte/Schule → Arbeit          3 : 1
# Einkauf → Einkauf und Services                           4 : 2
# private Erledigung → Einkauf und Services                5 : 2
# Bringen/Holen/Begleiten von Personen → Sonstiges         6 : 4
# Freizeitaktivität → Freizeitaktivität                    7 : 3
# nach Hause → Zuhause                                     8 : 0 
# Rückweg vom vorherigen Weg                               9 → Rückwege zuvor neu ausgewiesen
# anderer Zweck → Sonstiges                               10 : 4
# keine Angabe → Sonstiges                                99 : 4

reassignment = {1:1, 2:1, 3:1, 4:2, 5:2, 6:4, 7:3, 8:0, 10:4, 99:4}
data["Whyto"] = data["Whyto"].replace(reassignment)

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [48]:
# Ursprungszustände bestimmen
whyfrom = [] 
for i in range(len(data)):
    # erste Trips starten zuhause
    if data.at[i, "Trip_no"] == 1:
        whyfrom.append(0)
    # alle anderen vom Zielzustand des Trips zuvor
    else: 
        whyfrom.append(data.at[i-1, "Whyto"])
data["Whyfrom"] = whyfrom

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [49]:
data.head()

,ID,Trip_no,Type_day,Departure(hh:mm),Arrival(hh:mm),Departure,Arrival,Trip_duration,Whyto,Whyto_mop,Distance,Av_speed,Holiday,Work_status,W_ZWDE,W_ZWDP,W_ZWDF,Overnight,RegioStaR7,RegioStaR4,RegioStaRGem7,RegioStaRGem5,Mode,Whyfrom
0,1002822,1,1,5:50:00,5:55:00,350,355,5,1,1,0.19,2.28,0,1,7704,7704,7704,0,77,22,77,55,4,0
1,1002822,2,1,14:30:00,14:55:00,870,895,25,2,3,23.28,55.87,0,1,7704,601,7704,0,77,22,77,55,4,1
2,1002822,3,1,15:30:00,15:55:00,930,955,25,0,5,23.28,55.87,0,1,7704,7704,7704,0,77,22,77,55,4,2
3,1007211,1,1,10:00:00,10:19:00,600,619,19,2,3,3.80,12.00,0,7,7704,602,7704,0,77,22,77,55,2,0
4,1007211,2,1,10:29:00,10:35:00,629,635,6,2,3,1.90,19.00,0,7,501,7704,7704,0,77,22,77,55,2,2


In [50]:
# Whyfrom Spalte vor Whyto Spalte
cols_to_order = ["ID", "Trip_no", "Type_day", "Departure(hh:mm)", "Arrival(hh:mm)", "Departure", "Arrival", "Trip_duration", "Whyfrom", "Whyto"]
new_columns = cols_to_order + (data.columns.drop(cols_to_order).tolist())
data = data[new_columns]

In [51]:
data.drop(columns="Whyto_mop", inplace=True)

In [52]:
# Test Datensatz auf logische Konsistenz 
def test_dataset(data):
    err_first_trips = []
    err_trip_sequence = []
    err_trip_duration = []
    neg_stay_duration = []

    for i in range(len(data)-1):
        id_cur = data.at[i, "ID"]
        id_next = data.at[i+1, "ID"]
        tripno_cur = data.at[i, "Trip_no"]
        tripno_next = data.at[i+1, "Trip_no"]

        # Beginnt jedes Triptagebuch mit Trip Nummer 1 ? 
        if id_next != id_cur:
            if tripno_next != 1:
                err_first_trips.append(id_next)

        # Ist die Reihenfolge der Trips konsistent ?
        if id_next == id_cur:
            if tripno_next != tripno_cur + 1:
                err_trip_sequence.append(id_cur)

        # Entspricht Arrival dem Departure Wert + Tripdauer ?      
        if (data.at[i, "Arrival"] != data.at[i, "Departure"]  + data.at[i, "Trip_duration"]):
            if data.at[i, "Overnight"] == 0:
                err_trip_duration.append(id_cur)
            else: 
                if data.at[i, "Arrival"] != data.at[i, "Departure"]  + data.at[i, "Trip_duration"] - 1440:
                    err_trip_duration.append(id_cur)
        
        # Ist die Zeitfolge der Trips sinnvoll ?
        if (id_next == id_cur) & (data.at[i, "Arrival"] > data.at[i+1, "Departure"]):
                neg_stay_duration.append(id_cur)

    # Existieren Tripdauern < 0 ?
    neg_trip_duration = list(data[data["Trip_duration"] < 0]["ID"])
    
    # Wenn alle Listen leer sind, ist der Datensatz konsistent
    if (err_first_trips or err_trip_sequence or err_trip_duration or neg_stay_duration or neg_trip_duration):
        print("Fehler im Datensatz!")
        print("Anzahl problematischer IDs: \n err_first_trips = %d \n" \
              " err_trip_sequence = %d \n err_trip_duration = %d \n" \
              " neg_stay_duration = %d \n neg_trip_duration = %d"
              % (len(err_first_trips), len(err_trip_sequence), len(err_trip_duration), len(neg_stay_duration), len(neg_trip_duration))) 

    else: 
        print("Datensatz konsistent!")
    
    return [err_first_trips, err_trip_sequence, err_trip_duration, neg_stay_duration, neg_trip_duration]

In [53]:
faulty_trips = test_dataset(data)

Fehler im Datensatz!
Anzahl problematischer IDs: 
 err_first_trips = 0 
 err_trip_sequence = 0 
 err_trip_duration = 13 
 neg_stay_duration = 200 
 neg_trip_duration = 0


In [54]:
data[data["ID"].isin(faulty_trips[2])]

,ID,Trip_no,Type_day,Departure(hh:mm),Arrival(hh:mm),Departure,Arrival,Trip_duration,Whyfrom,Whyto,Distance,Av_speed,Holiday,Work_status,W_ZWDE,W_ZWDP,W_ZWDF,Overnight,RegioStaR7,RegioStaR4,RegioStaRGem7,RegioStaRGem5,Mode
25698,16656344,1,6,9:30:00,2:00:00,570,120,39,0,3,28.50,9995.00,0,1,7704,7704,701,1,73,11,75,53,4
33305,22998553,1,3,8:00:00,8:30:00,480,510,30,0,1,19.00,38.00,0,1,7704,7704,7704,0,77,22,77,55,2
33306,22998553,2,3,12:00:00,14:00:00,720,840,120,1,1,190.00,95.00,0,1,7704,7704,7704,0,77,22,77,55,2
33307,22998553,3,3,19:00:00,4:00:00,1140,240,197,1,0,712.50,9995.00,0,1,7704,7704,7704,1,77,22,77,55,2
36660,25560983,1,2,7:15:00,7:30:00,435,450,15,0,1,8.55,34.20,0,6,7704,7704,7704,0,76,21,76,54,2
36661,25560983,2,2,16:30:00,16:45:00,990,1005,15,1,0,8.55,34.20,0,6,7704,7704,7704,0,76,21,76,54,2
36662,25560983,3,2,18:30:00,18:35:00,1110,1115,5,0,3,2.85,34.20,0,6,7704,7704,714,0,76,21,76,54,2
36663,25560983,4,2,19:45:00,19:50:00,1185,1190,5,3,0,2.85,34.20,0,6,7704,7704,7704,0,76,21,76,54,2
36664,25560983,5,2,20:30:00,7:00:00,1230,420,34,0,3,28.50,9995.00,0,6,7704,7704,701,1,76,21,76,54,2
85218,59290211,1,5,5:20:00,5:50:00,320,350,30,0,1,19.00,38.00,0,1,7704,7704,7704,0,77,21,77,55,2


In [55]:
data[data["ID"].isin(faulty_trips[3])]

,ID,Trip_no,Type_day,Departure(hh:mm),Arrival(hh:mm),Departure,Arrival,Trip_duration,Whyfrom,Whyto,Distance,Av_speed,Holiday,Work_status,W_ZWDE,W_ZWDP,W_ZWDF,Overnight,RegioStaR7,RegioStaR4,RegioStaRGem7,RegioStaRGem5,Mode
3538,3291031,1,5,9:00:00,9:10:00,540,550,10,0,4,2.38,14.28,0,7,2202,2202,2202,0,73,11,76,54,3
3539,3291031,2,5,10:00:00,11:15:00,600,675,75,4,0,2.38,1.90,0,7,2202,2202,2202,0,73,11,76,54,3
3540,3291031,3,5,11:00:00,11:15:00,660,675,15,0,4,4.75,19.00,0,7,2202,2202,2202,0,73,11,76,54,3
3541,3291031,4,5,12:00:00,12:45:00,720,765,45,4,0,4.75,6.33,0,7,2202,2202,2202,0,73,11,76,54,3
3542,3291031,5,5,19:00:00,19:20:00,1140,1160,20,0,2,17.10,51.30,0,7,2202,2202,2202,0,73,11,76,54,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265079,97629572,3,7,20:00:00,21:30:00,1200,1290,90,3,0,76.00,50.67,0,7,2202,2202,2202,0,73,11,76,54,3
268343,99278451,1,5,5:00:00,5:30:00,300,330,30,0,1,19.95,39.90,0,1,2202,2202,2202,0,77,22,77,55,3
268344,99278451,2,5,14:10:00,15:40:00,850,940,90,1,0,19.95,13.30,0,1,2202,2202,2202,0,77,22,77,55,3
268345,99278451,3,5,15:30:00,15:40:00,930,940,10,0,4,4.75,28.50,0,1,2202,2202,2202,0,77,22,77,55,3


In [56]:
# vernachlässigen der Triptagebücher mit inkonsistenten Tripdauern
filt = data["ID"].isin(faulty_trips[2])
data = drop_by_filter(data, filt)

In [57]:
# vernachlässigen der Triptagebücher mit inkonsistenten Abfahrts und Ankunftszeiten
filt = data["ID"].isin(faulty_trips[3])
data = drop_by_filter(data, filt)

In [58]:
faulty_trips = test_dataset(data)

Datensatz konsistent!


In [59]:
# Auswertung Reihe für Reihe, Zuordnung der Trips über ID
def calc_aufenthalt(data):
    rows = data.shape[0]
    for i in range(rows):
        # letzter Trip der Person -> Aufenthalt = Zeit bis zum ersten Trip 
        if (i == rows-1) or (data.at[i + 1, "ID"] != data.at[i, "ID"]):
            # Beispiel: Wenn Wegnummer = 3 liegt Index des ersten Wegs 2 Schritte zurück
            index_first_trip = i - (data.at[i, "Trip_no"] - 1) 
            start_first_trip = data.at[index_first_trip, "Departure"]
            end_trip = data.at[i, "Arrival"]
            # Gesamtaufenthalt = 
            # Rest des Tages + Zeit bis zum ersten Trip
            if data.at[i, "Overnight"] == 0:          
                data.at[i, "Stay_duration"] = 1440 - end_trip + start_first_trip
            # Wenn letzte Fahrt nach 24:00
            else:
                data.at[i, "Stay_duration"] = start_first_trip - end_trip
        else:
            end_trip = data.at[i, "Arrival"]
            start_next_trip = data.at[i + 1, "Departure"]
            aufenthalt = start_next_trip - end_trip
            data.at[i, "Stay_duration"] = aufenthalt


In [60]:
# Berechnen der Aufenthaltsdauern
data["Stay_duration"] = pd.Series(dtype=int)
calc_aufenthalt(data)

In [61]:
data.head()

,ID,Trip_no,Type_day,Departure(hh:mm),Arrival(hh:mm),Departure,Arrival,Trip_duration,Whyfrom,Whyto,Distance,Av_speed,Holiday,Work_status,W_ZWDE,W_ZWDP,W_ZWDF,Overnight,RegioStaR7,RegioStaR4,RegioStaRGem7,RegioStaRGem5,Mode,Stay_duration
0,1002822,1,1,5:50:00,5:55:00,350,355,5,0,1,0.19,2.28,0,1,7704,7704,7704,0,77,22,77,55,4,515.0
1,1002822,2,1,14:30:00,14:55:00,870,895,25,1,2,23.28,55.87,0,1,7704,601,7704,0,77,22,77,55,4,35.0
2,1002822,3,1,15:30:00,15:55:00,930,955,25,2,0,23.28,55.87,0,1,7704,7704,7704,0,77,22,77,55,4,835.0
3,1007211,1,1,10:00:00,10:19:00,600,619,19,0,2,3.80,12.00,0,7,7704,602,7704,0,77,22,77,55,2,10.0
4,1007211,2,1,10:29:00,10:35:00,629,635,6,2,2,1.90,19.00,0,7,501,7704,7704,0,77,22,77,55,2,20.0


In [62]:
# 15 minütige Ankunfts- und Abfahrtsintervalle
data["Arrival_t"] = data["Arrival"].apply(lambda x: int(x/15))
data["Departure_t"] = data["Departure"].apply(lambda x: int(x/15))

In [63]:
# Stay_duration Zeile umpositionieren
cols_to_order = ["ID", "Trip_no", "Type_day", "Departure(hh:mm)", "Arrival(hh:mm)", "Departure_t", "Arrival_t", "Departure", "Arrival", "Trip_duration", "Whyfrom", "Whyto", "Stay_duration"]
new_columns = cols_to_order + (data.columns.drop(cols_to_order).tolist())
data = data[new_columns]

In [64]:
filt = data["Trip_no"] == 1
no_trip_diaries = len(data[filt])
no_trips = len(data)

print("Nach Aufbereitung des Datensatzes, verbleiben insgesamt %d Wege und %d Wegetagebücher." % (no_trips, no_trip_diaries))
print("Die durchschnittliche Anzahl an Trips pro Person beträgt: %s \n" % (no_trips/no_trip_diaries))

filt = (data["Type_day"] < 6) & (data["Holiday"] == 0)
data_we = data[filt]
no_trip_diaries_we = len(data_we[data_we["Trip_no"] == 1])
no_trips_we = len(data_we)

print("Davon %d Wege aus %d Wegetagebüchern an Werktagen." % (no_trips_we, no_trip_diaries_we))
print("Die durchschnittliche Anzahl an Trips pro Person beträgt: %f \n" % (no_trips_we/no_trip_diaries_we))

filt = (data["Type_day"] == 6) & (data["Holiday"] == 0)
data_sa = data[filt]
no_trip_diaries_sa = len(data_sa[data_sa["Trip_no"] == 1])
no_trips_sa = len(data_sa)

print("Davon %d Wege aus %d Wegetagebüchern an Samstagen." % (no_trips_sa, no_trip_diaries_sa))
print("Die durchschnittliche Anzahl an Trips pro Person beträgt: %f \n" % (no_trips_sa/no_trip_diaries_sa))

filt = (data["Type_day"] == 7) | (data["Holiday"] == 1)
data_so = data[filt]
no_trip_diaries_so = len(data_so[data_so["Trip_no"] == 1])
no_trips_so = len(data_so)

print("Davon %d Wege aus %d Wegetagebüchern an Sonn- oder Feiertagen." % (no_trips_so, no_trip_diaries_so))
print("Die durchschnittliche Anzahl an Trips pro Person beträgt: %f \n" % (no_trips_so/no_trip_diaries_so))


Nach Aufbereitung des Datensatzes, verbleiben insgesamt 269386 Wege und 85559 Wegetagebücher.
Die durchschnittliche Anzahl an Trips pro Person beträgt: 3.148540773033813 

Davon 199651 Wege aus 61184 Wegetagebüchern an Werktagen.
Die durchschnittliche Anzahl an Trips pro Person beträgt: 3.263124 

Davon 39910 Wege aus 12690 Wegetagebüchern an Samstagen.
Die durchschnittliche Anzahl an Trips pro Person beträgt: 3.144996 

Davon 29825 Wege aus 11685 Wegetagebüchern an Sonn- oder Feiertagen.
Die durchschnittliche Anzahl an Trips pro Person beträgt: 2.552418 



In [46]:
data_we.head()

,ID,Trip_no,Type_day,Departure(hh:mm),Arrival(hh:mm),Departure_t,Arrival_t,Departure,Arrival,Trip_duration,Whyfrom,Whyto,Stay_duration,Distance,Av_speed,Holiday,Work_status,W_ZWDE,W_ZWDP,W_ZWDF,Overnight,RegioStaR7,RegioStaR4,RegioStaRGem7,RegioStaRGem5,Mode
0,1002822,1,1,5:50:00,5:55:00,23,24,350,355,5,0,1,515.0,0.19,2.28,0,1,7704,7704,7704,0,77,22,77,55,4
1,1002822,2,1,14:30:00,14:55:00,58,60,870,895,25,1,2,35.0,23.28,55.87,0,1,7704,601,7704,0,77,22,77,55,4
2,1002822,3,1,15:30:00,15:55:00,62,64,930,955,25,2,0,835.0,23.28,55.87,0,1,7704,7704,7704,0,77,22,77,55,4
3,1007211,1,1,10:00:00,10:19:00,40,41,600,619,19,0,2,10.0,3.80,12.00,0,7,7704,602,7704,0,77,22,77,55,2
4,1007211,2,1,10:29:00,10:35:00,42,42,629,635,6,2,2,20.0,1.90,19.00,0,7,501,7704,7704,0,77,22,77,55,2


In [44]:
# Speichern der Daten
data_we.to_csv(r'C:\Users\user\Desktop\MiD\Aufbereitete Daten\Trips_processed_we.csv', header=True, index=False) 
data_sa.to_csv(r'C:\Users\user\Desktop\MiD\Aufbereitete Daten\Trips_processed_sa.csv', header=True, index=False) 
data_so.to_csv(r'C:\Users\user\Desktop\MiD\Aufbereitete Daten\Trips_processed_so.csv', header=True, index=False) 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\thoma\\Desktop\\MiD\\Aufbereitete Daten\\Trips_processed_we.csv'